Trabajo final para la asignatura APBD

Máster en Data Science y Big Data. CFP-Universidad de Sevilla

Inmaculada Perea Fernández

julio 2017


# Tabla de contenidos

1. [Descripción del problema](#descripcion)

2. [Inicio sesión Spark](#inicio)

3. [Lectura de datos y exploración](#exploracion)

4. [Preprocesado](#preprocesado)

    4.1. [Valores perdidos (missings)](#missings)
    
    4.2. [Conversión a tipo numérico](#numerico)
    
    4.3. [Escalado](#escalado)
    
    4.4. [Convertir features en vector (dataframe y label)](#features)
    
5. [Selección de variables (correlaciones, importancia de variables)](#seleccion)

6. [Modelos](#modelos)

    6.1. [Ajuste de parámetros](#ajuste)
    
    6.2. [Evaluación](#evaluacion)
       
7. [Cierre sesión spark](#cierre)


<a id='descripcion'></a>

# 1. Descripción del problema

El problema elegido para el trabajo es uno de los datasets de aprendizaje de kaggle, en el que el objetivo es predecir el precio de venta de viviendas a partir de sus características

Para más información sobre el problema ir a https://www.kaggle.com/c/house-prices-advanced-regression-techniques

El objetivo de este trabajo es practicar con las diferentes herramientas que *Spark* nos proporciona para el procesamiento distribuido de grandes cantidades de datos, y para la aplicación de técnicas de machine learning. La obtención de predicciones óptimas es un objetivo secundario aunque se valorará positivamente.

Las APIs de Spark más importantes para este proyecto serán la API de DataFrames y la API de ML para DataFrames. También tenemos a nuestra disposición la API RDDs para procesamiento a bajo nivel, y la API MLlib para RDDs.

<a id="inicio"></a>

# 2. Inicio sesión spark

In [2]:
import sys

# Ruta de la carpeta local Spark.
spark_path = 'C:/Users/inpf/spark-2.1.0-bin-hadoop2.7'

sys.path.append(spark_path + '/python')
sys.path.append(spark_path + '/python/lib/py4j-0.10.4-src.zip')

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName("evaluacion_APBD_InmaPerea").getOrCreate()

spark

Comprobamos el número de cores asignados

In [3]:
spark.sparkContext.defaultParallelism

4

<a id="exploracion"></a>

# 3. Lectura de datos y exploración

In [4]:
path = './data'
# en la pestaña storage sale el objeto que está cacheado en memoria
house_prices = spark.read.csv(path, header=True, inferSchema=True, nullValue="NA").cache()
house_prices.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- MSSubClass: integer (nullable = true)
 |-- MSZoning: string (nullable = true)
 |-- LotFrontage: integer (nullable = true)
 |-- LotArea: integer (nullable = true)
 |-- Street: string (nullable = true)
 |-- Alley: string (nullable = true)
 |-- LotShape: string (nullable = true)
 |-- LandContour: string (nullable = true)
 |-- Utilities: string (nullable = true)
 |-- LotConfig: string (nullable = true)
 |-- LandSlope: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Condition1: string (nullable = true)
 |-- Condition2: string (nullable = true)
 |-- BldgType: string (nullable = true)
 |-- HouseStyle: string (nullable = true)
 |-- OverallQual: integer (nullable = true)
 |-- OverallCond: integer (nullable = true)
 |-- YearBuilt: integer (nullable = true)
 |-- YearRemodAdd: integer (nullable = true)
 |-- RoofStyle: string (nullable = true)
 |-- RoofMatl: string (nullable = true)
 |-- Exterior1st: string (nullable = true)
 |-

In [5]:
cols = house_prices.columns
print("Número de variables = {0}".format(len(cols)))

Número de variables = 81


In [6]:
house_prices.show(3)

+---+----------+--------+-----------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+---------+
| Id|MSSubClass|MSZoning|LotFrontage|LotArea|Street|Alley|LotShape|LandContour|Utilities|LotConfig|LandSlope|Neighborhood|Condition1|Condition

<a id="preprocesado"></a>

# 4. Preprocesado

<a id="missings"></a>

## 4.1 Valores perdidos (missings)

Hay que indicar que los NA son los missings (nullValue="NA")

NA se ha usado tambien para rellenar los valores de las variables que no aplica, por ejemplo casas que no tienen garaje se ha completado con NA los metros cuadrados del garaje

In [7]:
from pyspark.sql.functions import col, count, sum as agg_sum
from pyspark.sql.types import *

def count_nulls(c):
    return agg_sum(col(c).isNull().cast(IntegerType())).alias(c)


In [8]:
# Definimos la operación para cada columna del DF
exprs = [count_nulls(c) for c in cols]

# Aplicamos todas las operaciones
missing = house_prices.agg(*exprs).first().asDict()
sorted([(v, k) for k, v in missing.items() if v > 0], reverse=True)

[(1453, 'PoolQC'),
 (1406, 'MiscFeature'),
 (1369, 'Alley'),
 (1179, 'Fence'),
 (690, 'FireplaceQu'),
 (259, 'LotFrontage'),
 (81, 'GarageYrBlt'),
 (81, 'GarageType'),
 (81, 'GarageQual'),
 (81, 'GarageFinish'),
 (81, 'GarageCond'),
 (38, 'BsmtFinType2'),
 (38, 'BsmtExposure'),
 (37, 'BsmtQual'),
 (37, 'BsmtFinType1'),
 (37, 'BsmtCond'),
 (8, 'MasVnrType'),
 (8, 'MasVnrArea'),
 (1, 'Electrical')]

<a id="numerico"></a>

## 4.2 Conversión a tipo numérico

<a id="escalado"></a>

## 4.3 Escalado

<a id="features"></a>

## 4.4 Convertir features en vector (dataframe y label)

<a id="seleccion"></a>

# 5. Selección de variables (correlaciones, importancia de variables)

<a id="modelos"></a>

# 6. Modelos

<a id="ajuste"></a>

## 6.1 Ajuste de parámetros

<a id="evaluacion"></a>

## 6.2 Evaluación

La evaluación del modelo final debe hacerse con un dataset diferente al de entrenamiento. 

Se recomienda automatizar procesos mediante el uso de pipelines y creación de funciones para evitar repetir código.

La métrica a usar para la evaluación de soluciones es *RMSLE*. Esta métrica no se encuentra dentro de las que ofrece el objeto *RegressionEvaluator* por lo que debemos definir nuestro propio evaluador.

In [9]:
'''
Código original tomado de:
https://gist.github.com/pvalienteverde

- Se ha añadido la fórmula rmsle para que trabaje con la API DataFrames.
'''

from pyspark.ml.evaluation import Evaluator
from math import sqrt
from operator import add
from pyspark.sql.functions import avg
from pyspark.sql.functions import log1p

class MyEvaluator(Evaluator):
    '''
    When a userID is predicted when it is not already trained (all userID  data is used on validation 
    group and none of them to train), prediction is nan,  so RegressionEvaluator returns Nan.
    To solve this we must change RegressionEvaluator by MiValidacion
    '''
    def __init__(self,predictionCol='prediction', targetCol='rating'):        
        super(MyEvaluator, self).__init__()
        self.predictionCol=predictionCol
        self.targetCol=targetCol
        
    def _evaluate(self, dataset):       
        error=self.rmse(dataset,self.predictionCol,self.targetCol)
        print ("Error: {}".format(error))
        return error
    
    def isLargerBetter(self):
        return False
    
    @staticmethod
    def rmse(dataset, predictionCol, targetCol):
        return sqrt(dataset.select(avg((dataset[targetCol] - dataset[predictionCol]) ** 2)).first()[0])
    
    @staticmethod
    def rmsle(dataset, predictionCol, targetCol):
        return sqrt(dataset.select(avg((log1p(dataset[targetCol]) - log1p(dataset[predictionCol])) ** 2)).first()[0])             

In [10]:
evaluator = MyEvaluator()

<a id="cierre"></a>

# 7. Cierre sesión spark

Liberamos recursos

In [14]:
spark.stop()